In [1]:
# mss, pyautogui

In [2]:
import numpy as np
import cv2
from mss import mss
from PIL import Image

import torch
import torchvision.transforms as transforms
from model.model import Yolov1
import autopy

import time

from model.utils import (
    load_checkpoint,
)
import torch.optim as optim

DEVICE = 'cuda'
LOAD_MODEL_FILE = "model/halo_model.pth.tar"
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0

In [3]:
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img):
        for t in self.transforms:
            img = t(img)
        return img

In [4]:
# transforms needed for the images
transform = Compose([transforms.Resize((448, 448)), transforms.ToTensor(),])

#create the model
model = Yolov1(split_size=7, num_boxes=2, num_classes=20).to(DEVICE)

optimizer = optim.Adam(
        model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )

# load the trained weights
load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

model.eval()

=> Loading checkpoint


Yolov1(
  (darknet): Sequential(
    (0): CNNBlock(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.1)
    )
    (1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (2): CNNBlock(
      (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.1)
    )
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): CNNBlock(
      (conv): Conv2d(192, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leakyrelu): LeakyReLU(negative_slope=0.1)
  

In [39]:
show = True
dbg = True


import model.utils as utils

#centering is how much of the screen to account for, from the center
centering = .5

sct = mss()

w, h = autopy.screen.size()

x_center = w/2
y_center = h/2

top = int(y_center - (y_center* centering))
left = int(x_center - (x_center* centering))
width = int(w)
height = int(h)
monitor = {'top': top, 'left': left, 'width': width, 'height': height}
resize = (640, 640)

while 1:
    if dbg:
        start = time.time()

    img = Image.frombytes('RGB', (width,height), sct.grab(monitor).rgb)
    img = img.resize(resize)
    img_transform = transform(img).unsqueeze(0).cuda()
    img = np.array(img)


    boxes = utils.cellboxes_to_boxes(model(img_transform))
    boxes = utils.non_max_suppression(boxes[0], iou_threshold=0.5, threshold=0.6, box_format="midpoint")
    if len(boxes) >0:
        primary, secondary = utils.pick_targets(img, boxes)
        print(boxes)

        if dbg:
            framerate = 1/(time.time() - start)

        # add framerate to the cv2 image
        cv2.putText(img, str(framerate), (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                       1, (255, 0, 0), 2, cv2.LINE_AA)
        for aimpoint in primary:
            print(aimpoint)
            cv2.circle(img,
                (int(aimpoint[0]), int(aimpoint[1])),
                radius=10,
                color=(255, 0, 0),
                thickness=5
            )


    if show:
        cv2.imshow('test', cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break




[[0.0, 0.6207857131958008, 0.5028741955757141, 0.4885517656803131, 0.042063526809215546, 0.12061677128076553]]
[321.83948516845703, 312.6731300354004]
[[0.0, 0.6207857131958008, 0.5028741955757141, 0.4885517656803131, 0.042063526809215546, 0.12061677128076553]]
[321.83948516845703, 312.6731300354004]
[[0.0, 0.6207857131958008, 0.5028741955757141, 0.4885517656803131, 0.042063526809215546, 0.12061677128076553]]
[321.83948516845703, 312.6731300354004]
[[0.0, 0.6207857131958008, 0.5028741955757141, 0.4885517656803131, 0.042063526809215546, 0.12061677128076553]]
[321.83948516845703, 312.6731300354004]
[[0.0, 0.6207857131958008, 0.5028741955757141, 0.4885517656803131, 0.042063526809215546, 0.12061677128076553]]
[321.83948516845703, 312.6731300354004]
[[0.0, 0.6207857131958008, 0.5028741955757141, 0.4885517656803131, 0.042063526809215546, 0.12061677128076553]]
[321.83948516845703, 312.6731300354004]
[[0.0, 0.6207857131958008, 0.5028741955757141, 0.4885517656803131, 0.042063526809215546, 0.120

In [48]:
import glob
# for img in glob.glob('C:/Users/blain/Documents\Git\AutoAim\model\halo_data/*.png'):
for img in glob.glob('C:/Users/blain\Documents\Git\AutoAim\model\halo_data/new/*'):

    img = Image.open(img)
    img = img.resize(resize)
    img_transform = transform(img).unsqueeze(0).cuda()
    img = np.array(img)


    boxes = utils.cellboxes_to_boxes(model(img_transform))
    boxes = utils.non_max_suppression(boxes[0], iou_threshold=0.5, threshold=0.4, box_format="midpoint")
    primary, secondary = utils.pick_targets(img, boxes)

    # add aim points to image
    for aimpoint in primary:
        print(aimpoint)
        cv2.circle(img,
            (int(aimpoint[0]), int(aimpoint[1])),
            radius=10,
            color=(255, 0, 0),
            thickness=5
        )
    cv2.imshow('test', cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR))

    if cv2.waitKey(0) & 0xFF == ord('q'):

        break
cv2.destroyAllWindows()

[305.9856414794922, 343.13629150390625]
[316.3752746582031, 333.47164154052734]
[311.5298080444336, 332.52105712890625]
[310.7513999938965, 329.5357894897461]


In [ ]:
import autopy
autopy.mouse.smooth_move(500, 500)

In [ ]:
import autopy
autopy.mouse.smooth_move(500, 500)

In [ ]:
import pyautogui
pyautogui.moveTo(100, 200, 0.25)   

In [ ]:

import math
import time
import random
import sys

TWO_PI = math.pi * 2.0


def sine_mouse_wave():
    """
    Moves the mouse in a sine wave from the left edge of
    the screen to the right.
    """
    width, height = autopy.screen.size()
    height /= 2
    height -= 10  # Stay in the screen bounds.

    for x in range(int(width)):
        y = int(height * math.sin((TWO_PI * x) / width) + height)
        autopy.mouse.move(x, y)
        time.sleep(.00000001)


sine_mouse_wave()

In [ ]:
import pyautogui
import pydirectinput
import autopy
import math
import time

TWO_PI = math.pi * 2.0

def sine_mouse_wave2():
    """
    Moves the mouse in a sine wave from the left edge of
    the screen to the right.
    """
    width, height = autopy.screen.size()
    height /= 2
    height -= 10  # Stay in the screen bounds.

    for x in range(int(width)):
        y = int(height * math.sin((TWO_PI * x) / width) + height)
        pydirectinput.moveTo(x, y, _pause=False)
        #time.sleep(.00000001)


sine_mouse_wave2()